# `nanoGPT`

## Install / Setup

### First Time Running

We need to install `ngpt` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install nanoGPT
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [ ]:
%%bash

[ $(python3 -c 'import ngpt; print(ngpt.__file__)') ] && export HAS_NGPT=1 || export HAS_NGPT=0
[ $HAS_NGPT ] || git clone https://github.com/saforem2/nanoGPT;
[ $HAS_NGPT ] || python3 nanoGPT/data/shakespeare_char/prepare.py
[ $HAS_NGPT ] || python3 -m pip install -e nanoGPT -vvv

## Post Install

In [ ]:
%load_ext autoreload
%autoreload 2

import ngpt
ngpt.__file__

## Build Trainer

In [ ]:
import os
from ezpz import setup_torch
from hydra.utils import instantiate
from ngpt.configs import get_config
from ngpt.trainer import Trainer

rank = setup_torch('DDP', seed=1234)
cfg = get_config(
    [
        'model=shakespeare',
        'train=shakespeare',
        'data=shakespeare',
        'train.log_interval=250',
        'train.eval_interval=2000',
        'optimizer=shakespeare',
        'train.dtype=bfloat16',
    ]
)
config = instantiate(cfg)
trainer = Trainer(config)

## Train Model

In [ ]:
trainer.train()

## Evaluate Model

In [ ]:
outputs = trainer.evaluate("what is the meaning of life?", num_samples=1)